In [1]:
import torch
from model import *
import numpy as np
import os
import random
import torch.optim as optim
import torch.nn as nn
from Myloader import *
import time
import torchvision.models as models
from torchmetrics.classification import MultilabelAveragePrecision
from ssl_encoder import *

C:\Users\User\anaconda3\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def evaluate(model, val_loader):
    model.eval()
    test_running_loss = 0.0
    test_total = 0

    with torch.no_grad():
        record_target_label = torch.zeros(1, 19).to(device)
        record_predict_label = torch.zeros(1, 19).to(device)
        for (test_imgs, test_labels, test_dicoms) in val_loader:
            test_imgs = test_imgs.to(device)
            test_labels = test_labels.to(device)
            test_labels = test_labels.squeeze(-1)

            test_output = model(test_imgs)
            loss = criterion(test_output, test_labels)

            test_running_loss += loss.item() * test_imgs.size(0)
            test_total += test_imgs.size(0)

            record_target_label = torch.cat((record_target_label, test_labels), 0)
            record_predict_label = torch.cat((record_predict_label, test_output), 0)


        record_target_label = record_target_label[1::]
        record_predict_label = record_predict_label[1::]

        metric = MultilabelAveragePrecision(num_labels=19, average="macro", thresholds=None)
        mAP = metric(record_predict_label, record_target_label.to(torch.int32))

    return mAP, test_running_loss, test_total

In [3]:
# set_seed(123)
#     weight_dir = ""
#     if not os.path.exists(weight_dir):
#         os.makedirs(weight_dir)

epochs = 100
batch_size = 32
num_classes = 19

weight_path = "weights/"

train_path = "data/MICCAI_long_tail_train.tfrecords"
train_index = "data/MICCAI_long_tail_train.tfindex"
val_path = "data/MICCAI_long_tail_val.tfrecords"
val_index = "data/MICCAI_long_tail_val.tfindex"
opt_lr = 1e-4
weight_decay = 1e-5
training = True
train_name = ""
val_name = ""

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
encoder = SSLEncoder(embDimension=num_classes).to(device)
# encoder = torch.load('ssl_backbone2.pth')
opt = optim.Adam(encoder.parameters(), lr=opt_lr, weight_decay = weight_decay)
train_loader = Myloader(train_path, train_index, batch_size, num_workers=0, shuffle=True)
val_loader = Myloader(val_path, val_index, batch_size, num_workers=0, shuffle=False)

criterion = nn.BCEWithLogitsLoss()

cuda


In [5]:
train_losses = []
test_losses = []

if training == True:
#         wandb.init(
#             project='chexpert mitigate bias',
#             name= train_wandb_name)
#         config = wandb.config
#         config.batch_size = batch_size
    max_map = 0
    total = 0
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(epochs):
        encoder.train()
        running_loss = 0.0
        start_time = time.time()
        count = 0

        for (imgs, labels, dicom_ids) in train_loader:
            encoder.zero_grad()
            opt.zero_grad()

            imgs = imgs.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(-1)

            with torch.autocast(device_type='cuda', dtype=torch.float16):
                output = encoder(imgs)
                # print('check')
                # print(output)
            loss = criterion(output, labels)

            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

            running_loss += loss.item() * imgs.size(0)
            count += imgs.size(0)

            if count != 0 and count % 1024 == 0 and total == 0:
                print(f"epoch {epoch}: {count}/unknown finished / train loss: {running_loss / count}")

            elif count != 0 and count % 10 == 0 and total != 0:
                print(f"epoch {epoch}: {count}/{total} (%.2f %%) finished / train loss: {running_loss / count}" % (count/total))

        total = count
        mAP, test_running_loss, test_total = evaluate(encoder, val_loader)
        
        train_losses.append(running_loss / count)
        test_losses.append(test_running_loss)
        
        if mAP > max_map:
            max_map = mAP
            torch.save({
                'model_state_dict': encoder.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
            }, f"{weight_path}/ssl_model_best.pt")
        if epoch % 10 == 0:
            torch.save({
                    'model_state_dict': encoder.state_dict(),
                    'optimizer_state_dict': opt.state_dict(),
                }, weight_path+"/{}epoch_ssl.pt".format(epoch))

        end_time = time.time()
        duration = end_time - start_time

        print(f"epoch {epoch} / mAP: {mAP} / test loss: {test_running_loss / test_total} / duration: {duration}")





epoch 0: 1024/unknown finished / train loss: 0.45958870090544224
epoch 0: 2048/unknown finished / train loss: 0.3854539804160595
epoch 0: 3072/unknown finished / train loss: 0.3589300929258267
epoch 0: 4096/unknown finished / train loss: 0.34597002738155425
epoch 0: 5120/unknown finished / train loss: 0.33829544074833395
epoch 0: 6144/unknown finished / train loss: 0.33084711090972024
epoch 0: 7168/unknown finished / train loss: 0.32766920181789566
epoch 0: 8192/unknown finished / train loss: 0.32384417892899364
epoch 0: 9216/unknown finished / train loss: 0.32119089230481124
epoch 0: 10240/unknown finished / train loss: 0.31876594135537745
epoch 0: 11264/unknown finished / train loss: 0.317232307554646
epoch 0: 12288/unknown finished / train loss: 0.31526755044857663
epoch 0: 13312/unknown finished / train loss: 0.31422261929569334
epoch 0: 14336/unknown finished / train loss: 0.3133797965544675
epoch 0: 15360/unknown finished / train loss: 0.31190085532143713
epoch 0: 16384/unknown f

epoch 0: 129024/unknown finished / train loss: 0.2878194751609708
epoch 0: 130048/unknown finished / train loss: 0.28773284164499346
epoch 0: 131072/unknown finished / train loss: 0.2877374272393354
epoch 0: 132096/unknown finished / train loss: 0.28762123072086726
epoch 0: 133120/unknown finished / train loss: 0.28760118085461167
epoch 0: 134144/unknown finished / train loss: 0.28757564562820753
epoch 0: 135168/unknown finished / train loss: 0.2875967978880825
epoch 0: 136192/unknown finished / train loss: 0.2874214319631289
epoch 0: 137216/unknown finished / train loss: 0.28740629109679094
epoch 0: 138240/unknown finished / train loss: 0.2873354548312448
epoch 0: 139264/unknown finished / train loss: 0.2873463251504718
epoch 0: 140288/unknown finished / train loss: 0.2871863439638358
epoch 0: 141312/unknown finished / train loss: 0.2871473621361066
epoch 0: 142336/unknown finished / train loss: 0.28713141208206366
epoch 0: 143360/unknown finished / train loss: 0.2870149863517976
epoc

epoch 1: 10400/182380 (0.06 %) finished / train loss: 0.2801826525651492
epoch 1: 10560/182380 (0.06 %) finished / train loss: 0.2800482292970022
epoch 1: 10720/182380 (0.06 %) finished / train loss: 0.2802105451697734
epoch 1: 10880/182380 (0.06 %) finished / train loss: 0.27986498155138073
epoch 1: 11040/182380 (0.06 %) finished / train loss: 0.27959328794824906
epoch 1: 11200/182380 (0.06 %) finished / train loss: 0.2797974893876484
epoch 1: 11360/182380 (0.06 %) finished / train loss: 0.27949141897785834
epoch 1: 11520/182380 (0.06 %) finished / train loss: 0.27953856583270764
epoch 1: 11680/182380 (0.06 %) finished / train loss: 0.2794109429398628
epoch 1: 11840/182380 (0.06 %) finished / train loss: 0.2794198347507296
epoch 1: 12000/182380 (0.07 %) finished / train loss: 0.2793326083421707
epoch 1: 12160/182380 (0.07 %) finished / train loss: 0.27904424479133205
epoch 1: 12320/182380 (0.07 %) finished / train loss: 0.2789367190429142
epoch 1: 12480/182380 (0.07 %) finished / trai

epoch 1: 28320/182380 (0.16 %) finished / train loss: 0.27610018923457735
epoch 1: 28480/182380 (0.16 %) finished / train loss: 0.27617962588085215
epoch 1: 28640/182380 (0.16 %) finished / train loss: 0.27630530352699023
epoch 1: 28800/182380 (0.16 %) finished / train loss: 0.2763110809855991
epoch 1: 28960/182380 (0.16 %) finished / train loss: 0.2761902238288637
epoch 1: 29120/182380 (0.16 %) finished / train loss: 0.27608661579561755
epoch 1: 29280/182380 (0.16 %) finished / train loss: 0.27615618044561374
epoch 1: 29440/182380 (0.16 %) finished / train loss: 0.2762079321819803
epoch 1: 29600/182380 (0.16 %) finished / train loss: 0.27602400884434986
epoch 1: 29760/182380 (0.16 %) finished / train loss: 0.27607263355485856
epoch 1: 29920/182380 (0.16 %) finished / train loss: 0.27625199868717293
epoch 1: 30080/182380 (0.16 %) finished / train loss: 0.276380492429784
epoch 1: 30240/182380 (0.17 %) finished / train loss: 0.2763776618967611
epoch 1: 30400/182380 (0.17 %) finished / tr

epoch 1: 46240/182380 (0.25 %) finished / train loss: 0.27606936923154085
epoch 1: 46400/182380 (0.25 %) finished / train loss: 0.27601295898700584
epoch 1: 46560/182380 (0.26 %) finished / train loss: 0.2759784724909006
epoch 1: 46720/182380 (0.26 %) finished / train loss: 0.27598529298828073
epoch 1: 46880/182380 (0.26 %) finished / train loss: 0.2759187442959372
epoch 1: 47040/182380 (0.26 %) finished / train loss: 0.27589714032452123
epoch 1: 47200/182380 (0.26 %) finished / train loss: 0.2760444706375316
epoch 1: 47360/182380 (0.26 %) finished / train loss: 0.2761317051947117
epoch 1: 47520/182380 (0.26 %) finished / train loss: 0.2760490665732811
epoch 1: 47680/182380 (0.26 %) finished / train loss: 0.2760660683848714
epoch 1: 47840/182380 (0.26 %) finished / train loss: 0.27602504628358476
epoch 1: 48000/182380 (0.26 %) finished / train loss: 0.2760037826001644
epoch 1: 48160/182380 (0.26 %) finished / train loss: 0.2759307588453705
epoch 1: 48320/182380 (0.26 %) finished / trai

...

epoch 16: 40320/182380 (0.22 %) finished / train loss: 0.2493952195440966
epoch 16: 40480/182380 (0.22 %) finished / train loss: 0.24937047130269965
epoch 16: 40640/182380 (0.22 %) finished / train loss: 0.24936647309562351
epoch 16: 40800/182380 (0.22 %) finished / train loss: 0.24930818418661754
epoch 16: 40960/182380 (0.22 %) finished / train loss: 0.24927588748978452
epoch 16: 41120/182380 (0.23 %) finished / train loss: 0.24925851717533304
epoch 16: 41280/182380 (0.23 %) finished / train loss: 0.24925127266220345
epoch 16: 41440/182380 (0.23 %) finished / train loss: 0.24937020302048982
epoch 16: 41600/182380 (0.23 %) finished / train loss: 0.24940029749503503
epoch 16: 41760/182380 (0.23 %) finished / train loss: 0.24935960342600885
epoch 16: 41920/182380 (0.23 %) finished / train loss: 0.24941678706926243
epoch 16: 42080/182380 (0.23 %) finished / train loss: 0.2494337057772698
epoch 16: 42240/182380 (0.23 %) finished / train loss: 0.2493884540078315
epoch 16: 42400/182380 (0.23

epoch 16: 57920/182380 (0.32 %) finished / train loss: 0.24885785985552802
epoch 16: 58080/182380 (0.32 %) finished / train loss: 0.2488233369654532
epoch 16: 58240/182380 (0.32 %) finished / train loss: 0.24882481670477888
epoch 16: 58400/182380 (0.32 %) finished / train loss: 0.2488479505744699
epoch 16: 58560/182380 (0.32 %) finished / train loss: 0.24887488898846621
epoch 16: 58720/182380 (0.32 %) finished / train loss: 0.2489092803374948
epoch 16: 58880/182380 (0.32 %) finished / train loss: 0.24887723329112582
epoch 16: 59040/182380 (0.32 %) finished / train loss: 0.24891288412941825
epoch 16: 59200/182380 (0.32 %) finished / train loss: 0.2489274982745583
epoch 16: 59360/182380 (0.33 %) finished / train loss: 0.2489518951212942
epoch 16: 59520/182380 (0.33 %) finished / train loss: 0.24893309988321796
epoch 16: 59680/182380 (0.33 %) finished / train loss: 0.24896186488722988
epoch 16: 59840/182380 (0.33 %) finished / train loss: 0.24892137406822196
epoch 16: 60000/182380 (0.33 %

epoch 16: 75520/182380 (0.41 %) finished / train loss: 0.24875032665618396
epoch 16: 75680/182380 (0.41 %) finished / train loss: 0.2487542345962363
epoch 16: 75840/182380 (0.42 %) finished / train loss: 0.2487936291827934
epoch 16: 76000/182380 (0.42 %) finished / train loss: 0.24877107291472586
epoch 16: 76160/182380 (0.42 %) finished / train loss: 0.24869614471407497
epoch 16: 76320/182380 (0.42 %) finished / train loss: 0.24866775579797396
epoch 16: 76480/182380 (0.42 %) finished / train loss: 0.24867894554736725
epoch 16: 76640/182380 (0.42 %) finished / train loss: 0.24866740918856325
epoch 16: 76800/182380 (0.42 %) finished / train loss: 0.2486366062425077
epoch 16: 76960/182380 (0.42 %) finished / train loss: 0.24862804203420072
epoch 16: 77120/182380 (0.42 %) finished / train loss: 0.24861532396548022
epoch 16: 77280/182380 (0.42 %) finished / train loss: 0.2485540031461242
epoch 16: 77440/182380 (0.42 %) finished / train loss: 0.24852411876902109
epoch 16: 77600/182380 (0.43 

epoch 16: 93120/182380 (0.51 %) finished / train loss: 0.2481905236686628
epoch 16: 93280/182380 (0.51 %) finished / train loss: 0.24817651828198245
epoch 16: 93440/182380 (0.51 %) finished / train loss: 0.24813802469878982
epoch 16: 93600/182380 (0.51 %) finished / train loss: 0.24815894513558118
epoch 16: 93760/182380 (0.51 %) finished / train loss: 0.24817758323592945
epoch 16: 93920/182380 (0.51 %) finished / train loss: 0.24820590995036357
epoch 16: 94080/182380 (0.52 %) finished / train loss: 0.24817023884986533
epoch 16: 94240/182380 (0.52 %) finished / train loss: 0.24819550710036112
epoch 16: 94400/182380 (0.52 %) finished / train loss: 0.2482251536846161
epoch 16: 94560/182380 (0.52 %) finished / train loss: 0.24822445499715468
epoch 16: 94720/182380 (0.52 %) finished / train loss: 0.24823688252048717
epoch 16: 94880/182380 (0.52 %) finished / train loss: 0.24822124870490866
epoch 16: 95040/182380 (0.52 %) finished / train loss: 0.2482234379307991
epoch 16: 95200/182380 (0.52

KeyboardInterrupt: 